<h1> Dataset acquisition </h1>

In [ ]:
import time
import csv
import threading
import ipywidgets as widgets
from witmotion import IMU
import os


class IMURecorder:
    def __init__(self, port="COM9", filename="imu_data.csv"):
        self.port = port
        self.filename = filename
        self.imu = None
        self.thread = None
        self.stop_flag = threading.Event()

    def start_acquisition(self, filename):
        """Starts data acquisition in a separate thread."""
        self.filename = filename
        if self.imu is None:
            self.imu = IMU(self.port)  # Open connection

        self.stop_flag.clear()
        self.thread = threading.Thread(target=self._acquire_data, daemon=True)
        self.thread.start()
        print(f"IMU acquisition started. Saving to {self.filename}...")

    def _acquire_data(self):
        """Internal method to acquire and save IMU data."""
        with open(self.filename, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Time (s)", "Accel X", "Accel Y", "Accel Z", "Gyro X", "Gyro Y", "Gyro Z"])  # Header

            start_time = time.time()

            while not self.stop_flag.is_set():
                current_time = time.time() - start_time
                a = self.imu.get_acceleration()
                g = self.imu.get_angular_velocity()

                if a and g:
                    writer.writerow([current_time, a[0], a[1], a[2], g[0], g[1], g[2]])

                time.sleep(0.1)  # Sampling interval

    def stop_acquisition(self):
        """Stops the data acquisition and closes the IMU connection."""
        if self.imu:
            self.stop_flag.set()
            self.thread.join()  # Wait for thread to exit
            self.imu.close()  # Properly close IMU connection
            self.imu = None
            print(f"Acquisition stopped. Data saved to {self.filename}")

directory = 'data'
if not os.path.exists(directory):
    os.makedirs(directory)

# Initialize the IMU recorder
recorder = IMURecorder(port='COM10')

# Filename input widget
filename_widget = widgets.Text(value="imu_data.csv", description="Filename:")

# Start acquisition button
start_button = widgets.Button(description="Start Acquisition")
start_button.on_click(lambda b: recorder.start_acquisition(os.path.join(directory, filename_widget.value)))

# Stop acquisition button
stop_button = widgets.Button(description="Stop Acquisition")
stop_button.on_click(lambda b: recorder.stop_acquisition())

# Display widgets in Jupyter
display(filename_widget, start_button, stop_button)
